<a href="https://colab.research.google.com/github/ttruong1000/MAT-494-Mathematical-Methods-for-Data-Science/blob/main/4_2_Spectral_Graph_Bipartitioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **4.2 - Spectral Graph Bipartitioning**

### **4.2.0 - Python Libraries for Spectral Graph Bipartitioning**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from itertools import cycle, islice
from imblearn import under_sampling as us
import time
import warnings

# **4.2.1 - Spectral Graph Bipartitioning**

Graph partition aims to find out a partition such that the cut (the total
number of edges between two disjoint sets of nodes) is minimized. For a
weighted graph $G =(V, E)$, given a bipartition of $V$ into disjoint graphs $V_1$ and $V_2$ ($V_1 \cup V_2 = V$), the cut between them can be defined as
\begin{equation*}
  \text{cut}(V_1, V_2, \ldots, V_k) = \sum_{i \in V_1, j \in V_2} M_{ij}
\end{equation*}
The definition of cut is easily extended to $k$ vertex subsets as
\begin{equation*}
  \text{cut}(V_1, V_2, \ldots, V_k) = \sum_{i < j} \text{cut}(V_i, V_j)
\end{equation*}
The classical graph bipartitioning problem is to find nearly equally-sized ver-
tex subset $V_i$, $V_2$ of $V$ such that $\text{cut}(V_1^*,V_2^*) = \min_{V_1, V_2} \text{cut}(V_1, V_2)$. For this purpose, let us define the partition vector $\mathbf{p}$ that captures this division.
\begin{equation*}
  p_i = \begin{cases}
    1 & \text{ for $i \in V_1$} \\
    -1 & \text{ for $i \in V_2$} \\
  \end{cases}
\end{equation*}
The cut can be characterized by the Rayleigh Quotient as follows.

##### Lemma 4.2.1.1 - Rayleigh Quotient

Given the Laplacian matrix $L$ of $G$ and a partition vector $\mathbf{p}$,
the Rayleigh Quotient
\begin{equation*}
  \frac{\mathbf{p}^TL\mathbf{p}}{\mathbf{p}^T\mathbf{p}} = \frac{1}{n}(4\text{cut}(V_1, V_2))
\end{equation*}

In practical applications, we also need an objective function to balance
cuts. Such an objective function can be formulated as follows. Let’s define a
diagonal matrix with $W$ where $w_{ii}$ is a weight for each vertex $i$. For a subset of vertices $V_l$ , define its weight to be weight $W_{V_l} = \displaystyle\sum_{i \in V_l} w_{ii}$. Now we try to balance subsets $V_1$ and $V_2$ in such a way that the following objective function, $Q(V_1, V_2)$, is minimized.
\begin{equation*}
  Q(V_1, V_2) = \frac{\text{cut}(V_1, V_2)}{W_{V_1}} + \frac{\text{cut}(V_1, V_2)}{W_{V_2}}
\end{equation*}

The minimization of $Q(V_1, V_2)$ favors partitions that have a small cut value
and are balanced because for two different partitions with the same cut value,
the above objective function value is smaller for the more balanced partition-
ing.

The objective function can be characterized by the Rayleigh Quotient of
the following generalized partition vector $\mathbf{q}$. Recall that all eigenvalues of $L$ are real and non-negative, and 0 is the smallest eigenvalue of $L$. For a given graph $G$, let $L$ and $W$ be its Laplacian and vertex weight matrices respectively. Let $\mathbf{e} = [1, 1, \ldots, 1]^T$ , $ν_1 = W_{V_1}$ and $ν_2 = W_{V_2}$. Then, the following result holds.

##### Theorem 4.2.2 - Seralized Partition Vectors

The serialized partition vector $\mathbf{q} = (q_i)$ defined as
\begin{equation*}
  q_i = \begin{cases}
    \sqrt{\frac{v_2}{v_1}} & \text{ for $i \in V_1$} \\
    -\sqrt{\frac{v_2}{v_1}} & \text{ for $i \in V_2$} \\
  \end{cases}
\end{equation*}
satisfies the following three properties.
1. $\mathbf{q}^TW\mathbf{e} = 0$, $\mathbf{q}^TW\mathbf{q} = v_1 + v_2$
2. $\frac{\mathbf{q}^TL\mathbf{q}}{\mathbf{q}^TW\mathbf{q}} = \frac{\text{cut}(V_1, V_2)}{v_1} + \frac{\text{cut}(V_1, V_2)}{v_2}$
3. The problem
\begin{equation*}
  \min_{q \neq 0} \frac{\mathbf{q}^TL\mathbf{q}}{\mathbf{q}^TW\mathbf{q}}, \quad \text{ subject to $\mathbf{q}^TW\mathbf{e} = 0$}
\end{equation*}
is solved when $\mathbf{q}$ is the eigenvector corresponding to the second smallest eigenvalue $\lambda_2$ of the generalized eigenvalue problem $L\mathbf{x} = \lambda W\mathbf{x}$.

Now we choose a $\text{weight}(i) = 1$ for all vertices $i$. This leads to the ratio-cut objective
\begin{equation*}
  \text{Ratio-Cut}(V_1, V_2) = \frac{\text{cut}(V_1, V_2)}{|V_1|} + \frac{\text{cut}(V_1, V_2)}{|V_2|}
\end{equation*}
One commonly used $W = \text{diag}(w_{ii})$ is to choose $w_{ii}$ to be the sum of the weights of edges incident on the node $i$, i.e., $w_{ii} = \displaystyle\sum_k E_{ik}$. This leads to the normalized cut criterion that was for image segmentation. Note that for this choice of vertex weights, the vertex weight matrix $W$ equals the degree matrix $D$, and weight
\begin{equation*}
  \sum_{j \in V_i} w_{jj} = \text{cut}(V_1, V_2) + \text{within}(V_i)
\end{equation*}
for $i = 1, 2$, where $\text{within}(V_i)$ is the sum of the weights of edges with both endpoints in $V_i$. Then, the normalized-cut objective function may be expressed as
\begin{equation*}
  \text{Normalized-Cut}(V_1, V_2) = \frac{\text{cut}(V_1, V_2)}{\displaystyle\sum_{i \in V_1} w_{ii}} + \frac{\text{cut}(V_1, V_2)}{\displaystyle\sum_{i \in V_2} w_{ii}} = 2 - S(V_1, V_2)
\end{equation*}
where $S(V_1, V_2) = \frac{\text{within}(V_1, V_2)}{\displaystyle\sum_{i \in V_1} w_{ii}} + \frac{\text{within}(V_1, V_2)}{\displaystyle\sum_{i \in V_2} w_{ii}}$. Note that $S(V_1, V_2)$ describes the strengths of associations within each partition. As a result, minimizing the normalized-cut is to maximize the proportion of edge weights that lie within each partition while balancing the cut.

# **4.2.2 - References**

1. MAT 494 Chapter 4 Notes